We want to compute $A^*_4$. In other words, we are looking for the coefficients of a convex polynomial
$p(x,y)=\sum_{i=0}^8 a_i x^i y^{8-i}$
that maximizes $\frac{a_4}{70}$ and satisfies $a_0=a_8=1$.

If $p(x, y)$ is an optimal solution, so is $p(y, x)$. We can therefore take $a_i = a_{8 - i}$ for $i=1,\ldots,7$.

In [10]:
import numpy as np
import sage_sdp  
from scipy.sparse import csc_matrix

# helper functions for symmetry reduction
#import group_morphisms 
import isotypical_decomposition

In [2]:
d = 4
a = tuple([polygen(QQ, 'a'+str(i)) for i in range(1, d+1)])
R.<x,y,u,v> = QQ[a]['x,y,u,v']
a = tuple(map(R, a))
a_repeated = (1,) + a[:-1] + a[::-1] + (1,)
show(a)
vars = [x,y,u,v]
q = sum([ a_repeated[i] * x^i * y^(2*d-i) * binomial(2*d, i)
         for i in range(2*d+1)])
obj = q.coefficient(x^d*y^d)
print "We want to maximize the variable `{obj}` and keep the form below convex."\
            .format(obj=obj)
show(q)
show(a)

(a1, a2, a3, a4)

We want to maximize the variable `70*a4` and keep the form below convex.


x^8 + 8*a1*x^7*y + 28*a2*x^6*y^2 + 56*a3*x^5*y^3 + 70*a4*x^4*y^4 + 56*a3*x^3*y^5 + 28*a2*x^2*y^6 + 8*a1*x*y^7 + y^8

(a1, a2, a3, a4)

In [3]:
Hq = jacobian(jacobian(q, (x, y)), (x, y))
H_quadratic_form = vector([u, v]) * Hq * vector([u, v])
print("q sos is equivalent to the form below being sos.")
show("H_quadratic_form = ", H_quadratic_form)

q sos is equivalent to the form below being sos.


'H_quadratic_form = ' 56*x^6*u^2 + 336*a1*x^5*y*u^2 + 840*a2*x^4*y^2*u^2 + 1120*a3*x^3*y^3*u^2 + 840*a4*x^2*y^4*u^2 + 336*a3*x*y^5*u^2 + 56*a2*y^6*u^2 + 112*a1*x^6*u*v + 672*a2*x^5*y*u*v + 1680*a3*x^4*y^2*u*v + 2240*a4*x^3*y^3*u*v + 1680*a3*x^2*y^4*u*v + 672*a2*x*y^5*u*v + 112*a1*y^6*u*v + 56*a2*x^6*v^2 + 336*a3*x^5*y*v^2 + 840*a4*x^4*y^2*v^2 + 1120*a3*x^3*y^3*v^2 + 840*a2*x^2*y^4*v^2 + 336*a1*x*y^5*v^2 + 56*y^6*v^2

# SDP formulation

`𝙷⎯𝚚𝚞𝚊𝚍𝚛𝚊𝚝𝚒𝚌⎯𝚏𝚘𝚛𝚖` is sos if and only if it can be written as $z^T Q z,$
where $Q$ is an $(2d-2)\times (2d-2)$ psd matrix and $z$ is the vector of monomials. (See below.)

In [4]:
#  H_quadratic_form sos if H_quadratic_form = z *
mons_xy = vector([x^i * y^(d-i-1) for i in range(d)])
mons = vector(list(u * mons_xy) + list(v * mons_xy))
show("Consider the monomials in (x,y,u,v): ", mons)
show("and the {i} x {i} matrix".format(i=len(mons)))

Q = sage_sdp.symb_matrix(len(mons), "Q")
show(Q[:5, :5], '...')

opt_variables = Q.variables() + a
print("New decision variables:")
show(opt_variables)

'Consider the monomials in (x,y,u,v): ' (y^3*u, x*y^2*u, x^2*y*u, x^3*u, y^3*v, x*y^2*v, x^2*y*v, x^3*v)

'and the 8 x 8 matrix'

[Q_11 Q_21 Q_31 Q_41 Q_51]
[Q_21 Q_22 Q_32 Q_42 Q_52]
[Q_31 Q_32 Q_33 Q_43 Q_53]
[Q_41 Q_42 Q_43 Q_44 Q_54]
[Q_51 Q_52 Q_53 Q_54 Q_55] '...'

New decision variables:


(Q_11,
 Q_21,
 Q_22,
 Q_31,
 Q_32,
 Q_33,
 Q_41,
 Q_42,
 Q_43,
 Q_44,
 Q_51,
 Q_52,
 Q_53,
 Q_54,
 Q_55,
 Q_61,
 Q_62,
 Q_63,
 Q_64,
 Q_65,
 Q_66,
 Q_71,
 Q_72,
 Q_73,
 Q_74,
 Q_75,
 Q_76,
 Q_77,
 Q_81,
 Q_82,
 Q_83,
 Q_84,
 Q_85,
 Q_86,
 Q_87,
 Q_88,
 a1,
 a2,
 a3,
 a4)

The identity $u^T\nabla^2 q u - z^TQz = 0$ is equivalent to a list of equalities involving the coefficients of $Q$ and the vector $a$ linearly  (one for each monomial). We report these linear equalities in the table below

In [5]:
residual = H_quadratic_form - mons * Q * mons
RR = QQ[opt_variables][x, y, u, v]
residual = RR(residual)
linear_eq = residual.coefficients()


table([residual.monomials(), [LatexExpr("{}=0".format(mi)) for mi in linear_eq]], 
      header_column=["monomial", "corresponding linear equality"])\
    .transpose()

monomial,corresponding linear equality
,-Q_44 + 56=0
,-2*Q_43 + 336*a1=0
,-Q_33 - 2*Q_42 + 840*a2=0
,-2*Q_32 - 2*Q_41 + 1120*a3=0
,-Q_22 - 2*Q_31 + 840*a4=0
,-2*Q_21 + 336*a3=0
,-Q_11 + 56*a2=0
,-2*Q_84 + 112*a1=0
,-2*Q_74 - 2*Q_83 + 672*a2=0
,-2*Q_64 - 2*Q_73 - 2*Q_82 + 1680*a3=0


# Construct a symmetry-adapted basis of monomials

In [6]:
# permutation group of symmetries
vars = [x, y, u, v]
symmetries =[[y,x,v,u],]
if d % 2 == 0:
    symmetries += [[-x,y,-u,v], [x,-y,-u,v]]
else:
    symmetries += [[-x,-y,-u,-v]]
symmetries_rho = map(lambda sym_i: matrix(QQ, jacobian(sym_i, vars)), symmetries)
G_sage = MatrixGroup(symmetries_rho)


print("The symmetries of the problem")
show(symmetries)
print "are represented by the group of", len(G_sage), "elements:"
show(G_sage)

The symmetries of the problem


[[y, x, v, u], [-x, y, -u, v], [x, -y, -u, v]]

are represented by the group of 16 elements:


Matrix group over Rational Field with 3 generators (
[0 1 0 0]  [-1  0  0  0]  [ 1  0  0  0]
[1 0 0 0]  [ 0  1  0  0]  [ 0 -1  0  0]
[0 0 0 1]  [ 0  0 -1  0]  [ 0  0 -1  0]
[0 0 1 0], [ 0  0  0  1], [ 0  0  0  1]
)

In [22]:

def get_irreducible_repr(G_sage):
    """Utility function to compute irreducible representations of a group using gap."""
    G_gap = gap.Group(map(lambda M: matrix(QQ, M), G_sage.gens()))
    irr_repr = []
    gap_irr_repr = gap.IrreducibleRepresentations(G_gap)
    characters = gap.Irr( G_gap )
    for irr in gap_irr_repr:            
        sage_irr = gap.UnderlyingRelation(irr)
        sage_irr = sage_irr.AsList()
        sage_irr = map(lambda u: list(u.AsList()),
                       sage_irr)  
        sage_irr = map(lambda elem_img: ((elem_img[0]), 
                                         elem_img[1].sage()), sage_irr)
        gens, vals = zip(*sage_irr)
        vals = map(lambda v: matrix(QQ, v), vals)
        gens_vals_dict = {G_sage(g): v for g,v in zip(gens, vals)}
        hom_irr = lambda g, gens_vals_dict=gens_vals_dict: gens_vals_dict[g]
        irr_repr.append(hom_irr)
    return irr_repr

irr_repr = get_irreducible_repr(G_sage)

In [23]:
# induced action on the vector of monomials z
def action_on_monomials(g):
    mons_permuted = mons.subs({vi: subi 
                               for vi, subi in zip(vars, g*vector(vars))})
    # construct permutation matrix
    col = np.array(range(len(mons)))
    basis_idx = {b: i for i,b in enumerate(mons)}
    row = [ basis_idx[m] if m in basis_idx else basis_idx[-m]  
           for m in mons_permuted]
    data = [1 if m in basis_idx else -1 for m in mons_permuted]
    return csc_matrix((data, (row, col))).todense()

print("Example of the action of G on the vector of monomials")
g = G_sage.random_element()
show(g, LatexExpr(r"\cdot"), vector(mons),
     " = ", matrix(QQ,action_on_monomials(g)) * mons)

Example of the action of G on the vector of monomials


[ 0  1  0  0]
[-1  0  0  0]
[ 0  0  0 -1]
[ 0  0  1  0] \cdot (y^3*u, x*y^2*u, x^2*y*u, x^3*u, y^3*v, x*y^2*v, x^2*y*v, x^3*v) ' = ' (x^3*v, -x^2*y*v, x*y^2*v, -y^3*v, -x^3*u, x^2*y*u, -x*y^2*u, y^3*u)

In [24]:
# compute a basis for the isotypical compononents of span(mons)
basis_iso = [isotypical_decomposition.\
             compute_basis_isotypical_comp(chi, action_on_monomials, G_sage) 
             for chi in irr_repr]

# compute a symmetry adapted basis
adapted_basis = []
for j, Vj in enumerate(basis_iso):
    if Vj is not None: 
        adapted_basis.append([ Vij * mons for Vij in Vj])
        
print("Symmetry adapted basis:")
show(adapted_basis)

Symmetry adapted basis:


[[(x*y^2*u + x^2*y*v, x^3*u + y^3*v)],
 [(y^3*u + x^3*v, x^2*y*u + x*y^2*v)],
 [(x*y^2*u - x^2*y*v, x^3*u - y^3*v)],
 [(y^3*u - x^3*v, x^2*y*u - x*y^2*v)]]

In [12]:
print("Let's build the semidefinite program")
size_Qs = map(lambda u: len(u[0]), adapted_basis)
Qs = [sage_sdp.symb_matrix(len(bi[0]), "Q"+str(i+1)) for i, bi in enumerate(adapted_basis)]
print("The matrix Q in the symmetry adapted basis has the following block diagonalization (with multiplicity):")
show([(len(bi), LatexExpr(r'\times'), Qi) for bi, Qi in zip(adapted_basis, Qs)])


Qs_vars = tuple(sum([list(Qi.variables()) for Qi in Qs], []))
decision_vars = Qs_vars + a
RR = QQ[decision_vars][x,y,u,v]
reduced_r = RR(H_quadratic_form  - \
       sum( sum( bij*Qi*bij for bij in bi ) for bi, Qi in zip(adapted_basis, Qs) ))

Let's build the semidefinite program
The matrix Q in the symmetry adapted basis has the following block diagonalization (with multiplicity):


[(
           [Q1_11 Q1_21]
1, \times, [Q1_21 Q1_22]
),
 (
           [Q2_11 Q2_21]
1, \times, [Q2_21 Q2_22]
),
 (
           [Q3_11 Q3_21]
1, \times, [Q3_21 Q3_22]
),
 (
           [Q4_11 Q4_21]
1, \times, [Q4_21 Q4_22]
)]

The identity $u^T\nabla^2 q u - z^TQz = 0$ is equivalent to a list of equalities involving the coefficients of $Q$ linearly  (one for each monomial $m$)
$$\langle A_m,  Q\rangle = 0.$$
We report these linear equalities in the table below

In [13]:
linear_eq = reduced_r.coefficients()
table([reduced_r.monomials(), [LatexExpr("{}=0".format(mi)) for mi in linear_eq]], 
      header_column=["monomial", "corresponding linear equality"])\
    .transpose()

monomial,corresponding linear equality
,-Q1_22 - Q3_22 + 56=0
,336*a1=0
,-2*Q1_21 - Q2_22 - 2*Q3_21 - Q4_22 + 840*a2=0
,1120*a3=0
,-Q1_11 - 2*Q2_21 - Q3_11 - 2*Q4_21 + 840*a4=0
,336*a3=0
,-Q2_11 - Q4_11 + 56*a2=0
,112*a1=0
,-2*Q1_21 - 2*Q2_21 + 2*Q3_21 + 2*Q4_21 + 672*a2=0
,1680*a3=0


Full dimensional description of the set

$\mathcal L := \{ v := (a, Q) \quad | \quad \text{$Q$ is psd and has the block diagonal structure described above and $a$ and $Q$ satisfy the linear equalities above} \}$
$\mathcal L := \{ v := (a, Q) \quad | \quad \text{$Q$  is psd and has the block diagonal structure described above and } A v = b \}$
$\mathcal L := \{ v := (a, Q) \quad | \quad v = v_0 + N \alpha, \alpha \in \mathbb R^k \},$
where $ker(A) =: \{N\alpha | \alpha \in \mathbb R^k\}$, and $k = dim(Ker(A))$

Let us construct the matrix $A$, the vector $b$, the vector $v_0$, and the matrix $N$.


In [14]:
def get_full_dim_description(linear_eq, vars, name='alpha'):
    A = matrix(QQ, jacobian(vector(SR, linear_eq), vector(vars)))
    b = vector(linear_eq) - A * vector(vars)
    v0 = A.solve_right(b)
    ker_A = matrix(SR, A.transpose().kernel().basis())
    alpha = tuple([var(name+str(i)) for i in range(1, ker_A.dimensions()[0]+1)])
    
    full_dim_vars = vector(v0) + ker_A.transpose() * vector(alpha)
    assert A * full_dim_vars - b == 0
    return full_dim_vars

full_dim_vars = get_full_dim_description(linear_eq, decision_vars, 'alpha')


In [15]:
original_to_full_dim = dict(zip(list(map(SR, decision_vars)), list(full_dim_vars)))
table([original_to_full_dim.keys(), original_to_full_dim.values()], 
      header_column=["original variables", "full dimensional description"]).transpose()

original variables,full dimensional description
,
,
,
,
,
,
,
,
,
,


In [16]:
def sub_list_matrices(list_matrices, subs):
    """Substitue according to the dict `subs`."""
    sub_map = lambda Mij: Mij.subs(subs)
    return list(map(lambda M: M.apply_map(sub_map), list_matrices))

Q_reduced = block_diagonal_matrix(sub_list_matrices(Qs, original_to_full_dim))
objective_reduced = original_to_full_dim[SR(a[-1])]
show("max ", objective_reduced)
show("s.t. the following matrix is psd")
show(Q_reduced)

'max ' 1/980*alpha1 + 1/490*alpha2 + 1/980*alpha3 - 13/980*alpha4 + 1/490*alpha5 + 1/980*alpha6 - 1/70*alpha8

's.t. the following matrix is psd'

[                                                                                           alpha1 + 28                                                                                                 alpha2|                                                                                                     0                                                                                                      0|                                                                                                     0                                                                                                      0|                                                                                                     0                                                                                                      0]
[                                                                                                alpha2                                                                                            alpha3 - 56|                                                                                                     0                                                                                                      0|                                                                                                     0                                                                                                      0|                                                                                                     0                                                                                                      0]
[-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------]
[                                                                                                     0                                                                                                      0|                                                                                                alpha4                                                                                                 alpha5|                                                                                                     0                                                                                                      0|                                                                                                     0                                                                                                      0]
[                                                                                                     0                                                                                                      0|                                                                                                alpha5                                                                                                 alpha6|                                                                                                     0                                                                                                      0|                                                                             

# Dual problem

In [17]:
Ds = [sage_sdp.symb_matrix(2, "D{}".format(i)) for i in range(4)]
D = block_diagonal_matrix(Ds)
show(D)

[D0_11 D0_21|    0     0|    0     0|    0     0]
[D0_21 D0_22|    0     0|    0     0|    0     0]
[-----------+-----------+-----------+-----------]
[    0     0|D1_11 D1_21|    0     0|    0     0]
[    0     0|D1_21 D1_22|    0     0|    0     0]
[-----------+-----------+-----------+-----------]
[    0     0|    0     0|D2_11 D2_21|    0     0]
[    0     0|    0     0|D2_21 D2_22|    0     0]
[-----------+-----------+-----------+-----------]
[    0     0|    0     0|    0     0|D3_11 D3_21]
[    0     0|    0     0|    0     0|D3_21 D3_22]

In [18]:
hessian = lambda pp: jacobian(jacobian(pp, (x,y)), (x,y))
dot = lambda u,v: vector(u) * vector(v)
mdot = lambda A, B: dot(A.list(), B.list())


lagrangian = mdot(D, Q_reduced) + objective_reduced
var_primal = Q_reduced.variables()
var_dual = D.variables()
lagrangian = QQ[var_dual][var_primal] (lagrangian)
show(LatexExpr(r"\mathcal L(\alpha, D) = "), lagrangian)

\mathcal L(\alpha, D) =  (D0_11 + 1/7*D2_21 - 1/7*D3_21 - 1/7*D3_22 + 1/980)*alpha1 + (2*D0_21 + 2/7*D2_21 + 12/7*D3_21 - 16/7*D3_22 + 1/490)*alpha2 + (D0_22 - 6/7*D2_21 - D2_22 + 6/7*D3_21 + 6/7*D3_22 + 1/980)*alpha3 + (D1_11 - 6/7*D2_21 - 78/7*D3_21 + 111/7*D3_22 - 13/980)*alpha4 + (2*D1_21 + 16/7*D2_21 - 2/7*D3_21 - 16/7*D3_22 + 1/490)*alpha5 + (D1_22 - 6/7*D2_21 + 6/7*D3_21 - 1/7*D3_22 + 1/980)*alpha6 + (D2_11 + D2_21 - D3_21 - D3_22)*alpha7 + (D3_11 - 12*D3_21 + 15*D3_22 - 1/70)*alpha8 + 28*D0_11 - 56*D0_22 - 28*D2_11

In [19]:
# Find a full rank description of the dual varialbe DQ
lagrang_coeffs = jacobian(lagrangian, var_primal)[0]
lagrang_coeffs = list(lagrang_coeffs)
full_dim_vars_dual = get_full_dim_description(lagrang_coeffs, var_dual, 'beta')
full_dim_vars_dual

(beta1 + 1/980, beta2 + 1/980, beta3 + 1/980, beta4 - 13/980, 5*beta1 + 5*beta2 + 1/3*beta4 + 1/980, -4*beta1 - 4*beta2 - 1/3*beta4 + 1/980, 7*beta1, -2*beta1 - 9*beta2 - 2/3*beta4, 6*beta1 + beta3, 6*beta1 + beta4 - 1/70, 3*beta1 - 5*beta2 - 1/3*beta4, 2*beta1 - 4*beta2 - 1/3*beta4)

In [20]:
original_to_full_dim_var = dict(zip(list(map(SR, var_dual)), list(full_dim_vars_dual)))
table([original_to_full_dim_var.keys(), original_to_full_dim_var.values()], 
      header_column=["original dual variables", "full dimensional description"]).transpose()

original dual variables,full dimensional description
,
,
,
,
,
,
,
,
,
,


In [21]:
D_reduced = block_diagonal_matrix(sub_list_matrices(Ds, original_to_full_dim_var))
show(D_reduced)

[                         beta1 + 1/980                          beta2 + 1/980|                                     0                                      0|                                     0                                      0|                                     0                                      0]
[                         beta2 + 1/980                          beta3 + 1/980|                                     0                                      0|                                     0                                      0|                                     0                                      0]
[-----------------------------------------------------------------------------+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------]
[                                     0                                      0|                        beta4 - 13/980  5*beta1 + 5*beta2 + 1/3*beta4 + 1/980|                                     0                                      0|                                     0                                      0]
[                                     0                                      0| 5*beta1 + 5*beta2 + 1/3*beta4 + 1/980 -4*beta1 - 4*beta2 - 1/3*beta4 + 1/980|                                     0                                      0|                                     0                                      0]
[-----------------------------------------------------------------------------+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------]
[                                     0                                      0|                                     0                                      0|                               7*beta1         -2*beta1 - 9*beta2 - 2/3*beta4|                                     0                                      0]
[                                     0                                      0|                                     0                                      0|        -2*beta1 - 9*beta2 - 2/3*beta4                        6*beta1 + beta3|                                     0                                      0]
[-----------------------------------------------------------------------------+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------]
[                                     0                                      0|                                     0                                      0|                                     0                                      0|                6*beta1 + beta4 - 1/70          3*beta1 - 5*beta2 - 1/3*beta4]
[                                     0                                      0|                                     0                                      0|                                     0                                      0|         3*beta1 - 5*beta2 - 1/3*beta4          2*beta1 - 4*beta2 - 1/3*beta4]

# use KKT conditions to transform the SDP to a system of polynomial equations¶


In [ ]:
# w = objective_reduced
w = var('omega')
R = QQ[Q_reduced.variables() + D_reduced.variables() + (w,)]

# KKT equations
KKT_eqn = (D_reduced * Q_reduced).list() + [w - objective_reduced]
KKT_eqn = list(set(KKT_eqn))
print("KKT equations = ")

table([list(map(R, KKT_eqn)), ["= 0" for _ in KKT_eqn]]).transpose()

Ideal generated by the KKT equations

In [ ]:
I = map(lambda p: R(p), KKT_eqn)
I = R*I
print("Ideal I is generated by %d equations in %d variables" % (len(I.gens()), len(R.gens())))

Solve the KKT equations by eliminating all variables except $\omega$


In [ ]:
I_a = I.elimination_ideal([v for v in R.gens() if SR(v) != w])
show(I_a)

In [ ]:
print("The ideal I_a has %d generator" % len(I_a.gens()))

In [ ]:
p = SR(I_a.gens()[0])
show(factor(p))

In [ ]:
minimal_poly = p.factor_list()[0][0]
minimal_poly /= minimal_poly.coefficient(w^3)
show(minimal_poly)
plot(minimal_poly)

In [ ]:
show(minimal_poly.roots())

In [ ]:
minimal_poly.roots(ring=RDF)